### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [2]:
! pip install -r cohere-embeddings-requirements.txt --quiet

### Provision the sample

This sample uses [`azd`](https://learn.microsoft.com/azure/developer/azure-developer-cli/), a bicep template, and a custom post-provision hook to provision the sample. The sample uses API keys for authentication.

1. Open a PowerShell command prompt in the cohere-embeddings folder.

1. Run `azd config set defaults.subscription <yourSubscriptionID>` to set the subscription if you have multiple Azure subscriptions.
1. Run `azd env new <your-environment-name>` to create a new `azd` environment. This environment will contain a set of environment variables used to provision the sample.
1. If you want to use an existing resource, set the corresponding `azd` environment variables before deployment:
   1. Existing Search resource:
      1. `azd env set AZURE_SEARCH_SERVICE <your-search-service-name>`
      1. `azd env set AZURE_SEARCH_SERVICE_LOCATION <your-search-service-location>`
      1. `azd env set AZURE_SEARCH_SERVICE_RESOURCE_GROUP <your-search-service-resource-group>`
      1. `azd env set AZURE_SEARCH_SERVICE_SKU <your-search-service-sku>`
      1. `azd env set AZURE_SEARCH_SERVICE_SEMANTIC_RANKER <your-semantic-ranker-sku>`
   1. You can use an existing AI Hub, AI Project, or Cohere Serverless Endpoint by setting the following environment variables:
      1. `azd env set AZUREAI_HUB_NAME <ai-hub-name>`
      1. `azd env set AZUREAI_PROJECT_NAME <ai-project-name>`
      1. `azd env set AZUREAI_SERVERLESS_ENDPOINT_NAME <cohere-serverless-endpoint-names>`
      1. `azd env set AZUREAI_MARKETPLACE_SUBSCRIPTION <cohere-marketplace-subscription-name>`
1. Run `azd provision`.
   1. Enter a development environment name.
   1. Enter a region for the deployment.

This step may take a few minutes to complete

### Retrieve environment variables after provisioning

The included `azd` bicep template saves all required environment variables for the notebook automatically.

In [3]:
# Load all environment variables from the azd deployment
import subprocess
from io import StringIO
from dotenv import load_dotenv
import os
result = subprocess.run(["azd", "env", "get-values"], stdout=subprocess.PIPE, cwd=os.getcwd())
load_dotenv(override=True, stream=StringIO(result.stdout.decode("utf-8")))

True

## Validate the indexer has completed successfully

An indexer runs in the background to chunk and vectorize all the sample documents. Validate that it has completed without any errors before trying to search the sample index. If there are any errors, they may be due to temporary throttling from the Azure OpenAI embeddings. Try re-running the indexer to resolve this issue.

In [4]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.identity import AzureCliCredential
from azure.core.credentials import AzureKeyCredential
from azure.mgmt.search import SearchManagementClient

credential = AzureCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID", None))
search_mgmt_client = SearchManagementClient(credential=credential, subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"))
key = search_mgmt_client.admin_keys.get(resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"), search_service_name=os.getenv("AZURE_SEARCH_SERVICE")).primary_key
search_indexer_client = SearchIndexerClient(endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"), credential=AzureKeyCredential(key))
status = search_indexer_client.get_indexer_status(name=os.getenv("AZURE_SEARCH_INDEXER"))
print(f"Status: {status.last_result.status}")

Status: success


### Perform a vector search by vectorizing your text query

Perform a vector search to find the most relevant images based on the text query.

Vector queries call [VectorizableTextQuery](https://learn.microsoft.com/python/api/azure-search-documents/azure.search.documents.models.vectorizabletextquery) to vectorize a query text string that's used to match against vectorized images created by AI Services. VectorizeableTextQuery uses the vectorizer defined in the index, which is a Cohere serverless endpoint hosted in Azure AI Studio.

In [5]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

# Generate text embeddings for the query  
query = "What is included in my Northwind Health Plus plan that is not in standard?"  
  
# Initialize the SearchClient  
search_client = SearchClient(endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"), index_name=os.getenv("AZURE_SEARCH_INDEX"), credential=AzureKeyCredential(key))  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="embedding")  

# Perform vector search  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["metadata_storage_path", "content"],
    top=3
)   

# Print the search results  
for result in results:  
    print(f"File: {os.path.basename(result['metadata_storage_path'])}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print("\n") 


File: Benefit_Options.pdf
Score: 0.8044232
Content: a variety of in-network providers, including primary care 
physicians, specialists, hospitals, and pharmacies. This plan does not offer coverage for emergency 
services, mental health and substance abuse coverage, or out-of-network services.

Comparison of Plans 
Both plans offer coverage for routine physicals, well-child visits, immunizations, and other preventive 
care services. The plans also cover preventive care services such as mammograms, colonoscopies, and 
other cancer screenings. 

Northwind Health Plus offers more comprehensive coverage than Northwind Standard. This plan offers 
coverage for emergency services, both in-network and out-of-network, as well as mental health and 
substance abuse coverage. Northwind Standard does not offer coverage for emergency services, mental 
health and substance abuse coverage, or out-of-network services. 

Both plans offer coverage for prescription drugs. Northwind Health Plus offers a wid

## Perform a hybrid search with semantic reranking

Combine the vector search with a text search for higher accuracy

In [6]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import HybridSearch

# Generate text embeddings for the query  
query = "What is included in my Northwind Health Plus plan that is not in standard?"  
  
# Initialize the SearchClient  
search_client = SearchClient(endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"), index_name=os.getenv("AZURE_SEARCH_INDEX"), credential=AzureKeyCredential(key))  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="embedding")  

# Perform vector search  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    semantic_query=query,
    hybrid_search=HybridSearch(max_text_recall_size=50),
    select=["metadata_storage_path", "content"],
    semantic_configuration_name='semantic-config',
    top=3
)   

# Print the search results  
for result in results:  
    print(f"File: {os.path.basename(result['metadata_storage_path'])}")
    print(f"Semantic score: {result['@search.reranker_score']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print("\n") 

File: Northwind_Standard_Benefits_Details.pdf
Semantic score: 2.844975471496582
Score: 0.027364110574126244
Content: includes counselling, psychotherapy, 

and other treatments related to mental health and substance abuse.  



Out-of-Network Services: The Northwind Standard plan does not cover any services that are 

provided by a provider that is not part of the Northwind Health network. This includes 

doctors, hospitals, and other healthcare providers who are not part of the Northwind Health 

network.  

Tips 

When selecting a healthcare plan, it is important to be aware of the exclusions in the plan. 

Here are some tips to help you understand the exclusions in the Northwind Standard plan:  

1. Understand the types of services that are not covered by the Northwind Standard plan. Be 

sure to familiarize yourself with the list of exclusions and make sure that any services you 

might require are covered.  

2. If you require emergency services, be sure to check with your provide